## 1 - Open source embedding model exploration

Exploring here other "open source" embedding models. 

For this task we will use MTEB that compares 100+ text (and image) embedding models across 1000, languages depending on metrics, languages, tasks, and task types:

https://huggingface.co/spaces/mteb/leaderboard

MTEB classe les modèles en fonction de  8 types de tâches  : ￼

1.	Bitext Mining : trouver des phrases correspondantes dans deux langues différentes.

2.	Classification : attribuer des catégories aux textes.

3.	Clustering : regrouper des textes similaires.

4.	Pair Classification : déterminer si deux textes sont similaires.

5.	Reranking : ordonner une liste de textes en fonction de leur pertinence par rapport à une requête.

6.	Retrieval : retrouver des documents pertinents pour une requête donnée.

7.	Semantic Textual Similarity (STS) : mesurer la similarité sémantique entre deux textes.

8.	Summarization : évaluer la qualité des résumés générés automatiquement. ￼


Critères de choix pour le modèle:
- Filtrer sur domaine juridique + langue Fr
- Performances en terme de Retrieval
- Nombre de tokens maximal à partir de 8191 en raison de la longeur de nos chunks (articles de loi dont la taille très variables)
- Taile du modele (en prenant le meilleur modele qui répond a tous les critères mais aussi un modele de 1B ou 0.5B)

Résultats de la selection en fonction des critères (classement en fonction des score sur Retreival):

- Catégorie 1 - modèles propriétaires: 

1 - [voyage-3](https://blog.voyageai.com/2024/09/18/voyage-3/) est le meilleur arrive en premier mais modele proprio (score de 85 en retreival) 

2 - ensuite vient celui de google [gemini-embedding-exp-03-07](https://developers.googleblog.com/en/gemini-embedding-text-model-now-available-gemini-api/) - (?B) -  max token 8192 - score global 74  - score retreival 67.71



- Catégorie 2 - modèles open source: 

1 - [inf-retriever-v1](https://huggingface.co/infly/inf-retriever-v1) (7B )  - score 73.89 mais aucun score sur STS ! sa version 1B est plus interessante car drop non significatif dans les perfs (72.14 )  [inf-retriever-v1-1.5b](https://huggingface.co/infly/inf-retriever-v1-1.5b)

2 - [SFR-Embedding-Mistral](https://huggingface.co/Salesforce/SFR-Embedding-Mistral) 7B  - max token 32768 - score 68.46

3 - [snowflake-arctic-embed-l-v2.0](https://huggingface.co/Snowflake/snowflake-arctic-embed-l-v2.0) - 0.5 B - max token 8192 - score 65

En intégrant le reranking au retreival : 

1 - [SFR-Embedding-Mistral](https://huggingface.co/Salesforce/SFR-Embedding-Mistral) - score global 84

2 - [snowflake-arctic-embed-l-v2.0](https://huggingface.co/Snowflake/snowflake-arctic-embed-l-v2.0) 80 en modele leger (0.5)


Remarque : Tous ces modèles semblent supérieurs aux perfs de [text-embedding-3-large](https://openai.com/index/new-embedding-models-and-api-updates/)  qui en retreival a un score de 59.27

Deux choix possible au regard du drop dans les perfs : 
- inf-retreiver-v1 (7B) en production endpoint sur HF 
- ou tenter le inf-retreiver-v1 en 1.5B sinon le modele de snowflake

In [ ]:
# 2. Sentence‑Transformers gère tout le reste
!pip install --upgrade sentence-transformers

In [ ]:
!pip install --upgrade pytorch

## 2 - Test loading inf-retriever-v1-1.5b

In [ ]:
import os
import torch
import numpy as np

In [11]:
import json, torch
from sentence_transformers import SentenceTransformer

# ----- 1) Choix de l'appareil (MPS pour Mac M1/M2) -----
device = "mps" if torch.backends.mps.is_available() else "cpu"


In [16]:
# ----- 2) Chargement du modèle prêt à l'emploi -----
model = SentenceTransformer(
    "infly/inf-retriever-v1-1.5b",   # ↙ modèle E5 + pooling
    device=device, 
    token=os.getenv("HF_token")
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [18]:
# ----- 3) Lecture du JSON source -----
with open("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_chunks/chunks_aml_5_strat_2.json", encoding="utf-8") as f:
    docs = json.load(f)

In [ ]:
# ----- 4) Génération des embeddings -----
for doc in docs:
    emb = model.encode(
        doc["page_content"],
        normalize_embeddings=True              # distance cosine déjà ok
    ).tolist()                                 # .tolist() pour JSON

    doc["embedding"] = emb                    # ajoute le vecteur
    # petite trace du modèle et de la dimension
    doc.setdefault("metadata", {})["embedding_model_name"] = (
        "infly/inf-retriever-v1-1.5b"
    )

In [ ]:
# ----- 5) Sauvegarde -----
with open("documents_with_embeddings.json", "w", encoding="utf-8") as f:
    json.dump(docs, f, ensure_ascii=False, indent=2)

print("✅  Embeddings ajoutés dans documents_with_embeddings.json")

Conclusion : 
- Working : ok
- Result : None
- Taking time to execute.

## 2 - Testing the most efficient model : snowflake-arctic-embed-l-v2.0 (0.5 B)

 Even if this model is ranked at the 5th position it is the lighetest and most efficient one without a significant drop in performance (only 0.5 B param)

In [1]:
import os
import torch
import numpy as np

In [ ]:
from sentence_transformers import SentenceTransformer

model_05B = SentenceTransformer(
    "infly/inf-retriever-v1-1.5b",   # ↙ modèle E5 + pooling
    device=device, 
    token=os.getenv("HF_token")
)

In [2]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
tokenizer= AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large-instruct",token =os.getenv('HF_token'))
model= AutoModel.from_pretrained("intfloat/multilingual-e5-large-instruct", token=os.getenv('HF_token'))

In [ ]:
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

task = 'Given a web search query, retrieve relevant passages that answer the query'
queries = [
    get_detailed_instruct(task, 'how much protein should a female eat'),
    get_detailed_instruct(task, '南瓜的家常做法')
]
documents = [
    "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day...",
    "1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法:..."
]
input_texts = queries + documents

In [ ]:
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

NameError: name 'tokenizer' is not defined

In [ ]:
with torch.no_grad():
    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

Définir la fonction de pooling :
Cette fonction effectue une moyenne pondérée par le masque d’attention.

In [ ]:
import torch.nn.functional as F

embeddings = F.normalize(embeddings, p=2, dim=1)

In [ ]:
embeddings

tensor([[ 0.0202,  0.0112, -0.0451,  ..., -0.0266, -0.0423,  0.0389],
        [ 0.0486,  0.0594,  0.0040,  ..., -0.0214, -0.0263,  0.0198],
        [ 0.0198, -0.0023, -0.0366,  ..., -0.0368, -0.0272,  0.0254],
        [ 0.0461,  0.0436,  0.0091,  ..., -0.0155, -0.0181,  0.0149]])

In [ ]:
similarity = torch.matmul(embeddings[0], embeddings[1].T)

/var/folders/ty/2g_hgfks4nl_84mz496fmn600000gn/T/ipykernel_82697/1230817599.py:1: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3729.)
  similarity = torch.matmul(embeddings[0], embeddings[1].T)


In [ ]:
embedding_list = embeddings.tolist()  
embedding_array = embeddings.cpu().numpy()  # to tbl NumPy

In [ ]:
embedding_array

array([[ 0.02018598,  0.01120441, -0.04514236, ..., -0.02658733,
        -0.04230955,  0.03893524],
       [ 0.04857629,  0.05938283,  0.00398616, ..., -0.02135831,
        -0.0263372 ,  0.01975931],
       [ 0.01981018, -0.00233509, -0.03661876, ..., -0.03679116,
        -0.02720564,  0.02541652],
       [ 0.04608595,  0.04364255,  0.00913067, ..., -0.01546213,
        -0.01805959,  0.01493509]], dtype=float32)

We choose to use the 5th one (multilingual-e5-large-instrc) since it performing well globally and still good on retreival , reranking and STS.
